In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Applying combination of augmentation functions to check roughly how it's applying on original tiles

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import apply_brightness_shift, apply_channel_shift, random_channel_shift, random_brightness
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define Image Directory
image_dir = "/content/drive/MyDrive/test/class_a"
augmented_dir = "/content/drive/MyDrive/augmented_images"
os.makedirs(augmented_dir, exist_ok=True)

# Load Images and Preprocess
def load_images(image_dir):
    images = []
    filenames = sorted(os.listdir(image_dir))

    for filename in filenames:
        img_path = os.path.join(image_dir, filename)

        # Read the image
        img = cv2.imread(img_path)

        # Check if the image is loaded properly
        if img is None:
            print(f"Skipping {filename}: Unable to load image.")
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (256, 256)) / 255.0
        images.append((filename, img))

    print(f"Loaded {len(images)} images successfully.")
    return images


image_data = load_images(image_dir)
print(f"Loaded {len(image_data)} images.")

# Augmentation Functions (Convert NumPy to Tensor)
# Apply Brightness Shift
def augment_brightness(image, delta=0.2):
    return tf.image.adjust_brightness(image, delta).numpy() if isinstance(image, tf.Tensor) else tf.image.adjust_brightness(tf.convert_to_tensor(image), delta).numpy()

# Apply Channel Shift
def augment_channel_shift(image, intensity=10.0):
    return apply_channel_shift(image, intensity) if isinstance(image, np.ndarray) else apply_channel_shift(image.numpy(), intensity)

# Apply Random Channel Shift
def augment_random_channel_shift(image, max_delta=0.2):
    return random_channel_shift(image, max_delta) if isinstance(image, np.ndarray) else random_channel_shift(image.numpy(), max_delta)

# Apply Random Brightness (Final Step)
def augment_random_brightness(image, delta=0.3):
    return tf.image.adjust_brightness(image, delta).numpy() if isinstance(image, tf.Tensor) else tf.image.adjust_brightness(tf.convert_to_tensor(image), delta).numpy()



# Augmentation Combinations
augmentation_combinations = {
    "brightness_shift": augment_brightness,
    "channel_shift": augment_channel_shift,
    "random_channel_shift": augment_random_channel_shift,
    "random_brightness": augment_random_brightness,
    "brightness_and_channel": lambda img: augment_channel_shift(augment_brightness(img)),
    "brightness_and_random_channel": lambda img: augment_random_channel_shift(augment_brightness(img)),
    "channel_and_random_brightness": lambda img: augment_random_brightness(augment_channel_shift(img)),
    "all_combined": lambda img: augment_random_brightness(augment_random_channel_shift(augment_brightness(img)))
}

# Apply Augmentations and Save Images
for aug_name, aug_function in augmentation_combinations.items():
    aug_path = os.path.join(augmented_dir, aug_name, "class_a")
    os.makedirs(aug_path, exist_ok=True)

    for filename, img in image_data:
        augmented_img = aug_function(img)
        augmented_img = np.clip(augmented_img, 0, 1)
        save_path = os.path.join(aug_path, filename)
        cv2.imwrite(save_path, cv2.cvtColor((augmented_img * 255).astype(np.uint8), cv2.COLOR_RGB2BGR))

    print(f"Augmented images saved in: {aug_path}")

# Applying 3 augmentation technique processing for artifacts removal/minimizing methods of Random channel shift, Brightness shift and both Brightness with random channel

In [ ]:
import os
import random
import numpy as np
import cv2
import tensorflow as tf
from PIL import Image
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import apply_channel_shift

# Define Paths
base_dir = "/content/drive/MyDrive"
class_paths = {
    "train_class_a": os.path.join(base_dir, "train", "class_a"),
    "train_class_b": os.path.join(base_dir, "train", "class_b"),
    "val_class_a": os.path.join(base_dir, "val", "class_a"),
    "val_class_b": os.path.join(base_dir, "val", "class_b"),
}

# Augmentation Settings
augment_factor = 3

# Define Augmentation Functions
def augment_brightness(image, delta=0.2):
    """Adjust brightness"""
    return tf.image.adjust_brightness(image, delta).numpy()

def augment_random_channel_shift(image, max_delta=0.2):
    """Apply random channel shift"""
    return tf.image.random_saturation(image, 1 - max_delta, 1 + max_delta).numpy()

def augment_brightness_random_channel(image):
    """Apply brightness and random channel shift together"""
    img = augment_brightness(image)
    return augment_random_channel_shift(img)

# Augmentations Dictionary
augmentations = {
    "random_channel_shift": augment_random_channel_shift,
   # "brightness_shift": augment_brightness,
   # "brightness_random_channel": augment_brightness_random_channel
}

# Apply Augmentations
for cls, cls_path in class_paths.items():
    print(f"Processing {cls} augmentation...")

    # Check if directory exists before processing
    if not os.path.exists(cls_path):
        print(f"Skipping {cls}: Directory not found -> {cls_path}")
        continue

    images = [img for img in os.listdir(cls_path) if img.endswith(".png")]

    for aug_name, aug_function in augmentations.items():
        augmented_dir = f"{cls_path}_{aug_name}_augmented"
        os.makedirs(augmented_dir, exist_ok=True)

        for img_name in tqdm(images, desc=f"Augmenting {cls} with {aug_name}"):
            img_path = os.path.join(cls_path, img_name)
            image = Image.open(img_path)
            image = image.convert("RGB")

            # Convert image to NumPy array and normalize
            image_np = np.array(image) / 255.0

            # Save original image
            image.save(os.path.join(augmented_dir, img_name))

            # Generate augmentations
            for i in range(augment_factor):
                augmented_img = aug_function(image_np)
                augmented_img = np.clip(augmented_img, 0, 1)
                augmented_img = (augmented_img * 255).astype(np.uint8)

                # Save augmented image
                aug_img_name = f"{os.path.splitext(img_name)[0]}_{aug_name}_aug_{i}.png"
                aug_img_path = os.path.join(augmented_dir, aug_img_name)
                Image.fromarray(augmented_img).save(aug_img_path)

        print(f"Augmented images saved in: {augmented_dir}")

print("Data augmentation completed successfully!")


Processing train_class_a augmentation...
Skipping train_class_a: Directory not found -> /content/drive/MyDrive/train/class_a
Processing train_class_b augmentation...
Skipping train_class_b: Directory not found -> /content/drive/MyDrive/train/class_b
Processing val_class_a augmentation...
Skipping val_class_a: Directory not found -> /content/drive/MyDrive/val/class_a
Processing val_class_b augmentation...
Skipping val_class_b: Directory not found -> /content/drive/MyDrive/val/class_b
✅ Data augmentation completed successfully!


# EfficientnetB0 model training for the augmented dataset of Channel shift method

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Normalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import os
import shutil

# Disable Mixed Precision (use float32 everywhere)
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('float32')

# Copy dataset from Google Drive to Colab RAM (Faster than streaming)
os.makedirs('/content/dataset', exist_ok=True)
if not os.path.exists('/content/dataset/train'):
    print("Copying dataset to Colab RAM...")
    shutil.copytree('/content/drive/MyDrive/train', '/content/dataset/train')
    shutil.copytree('/content/drive/MyDrive/val', '/content/dataset/val')
    shutil.copytree('/content/drive/MyDrive/test', '/content/dataset/test')

# Define dataset directories
train_dir = '/content/dataset/train'
val_dir = '/content/dataset/val'
test_dir = '/content/dataset/test'

# Define image dimensions and batch size
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 128
AUTOTUNE = tf.data.AUTOTUNE

# Load dataset using `tf.data`
raw_train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)
class_names = raw_train_dataset.class_names

# Define Data Augmentation Layer
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomBrightness(0.1),
])

# Define normalization layer
normalization_layer = Normalization()

# Function to convert images to float32
def cast_to_float32(image, label):
    return tf.cast(image, tf.float32), label

# Function to apply augmentation + normalization
def preprocess_image(image, label):
    image = data_augmentation(image)
    image = normalization_layer(image)
    return image, label

# Convert dataset images to float32 before applying normalization
normalized_train_dataset = raw_train_dataset.map(cast_to_float32)

# Adapt Normalization Layer using training data (only on images)
normalization_layer.adapt(normalized_train_dataset.map(lambda x, y: x))

# Apply augmentation & normalization to dataset
train_dataset = normalized_train_dataset.map(preprocess_image).prefetch(buffer_size=AUTOTUNE)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
).map(cast_to_float32).map(preprocess_image).prefetch(buffer_size=AUTOTUNE)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
).map(cast_to_float32).map(preprocess_image).prefetch(buffer_size=AUTOTUNE)

# Load Pre-trained EfficientNetB0 with Correct Input Shape
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add Custom Classification Layers
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.005))(x)
x = BatchNormalization()(x)
x = Dropout(0.6)(x)  # Dropout Increased
x = Dense(256, activation='relu', kernel_regularizer=l2(0.005))(x)
x = Dropout(0.6)(x)

# Use `len(class_names)`
outputs = Dense(len(class_names), activation='softmax')(x)

# Define the Model
model = Model(inputs=base_model.input, outputs=outputs)

# Freeze the First 120 Layers (EfficientNetB0 has ~236 layers)
for layer in base_model.layers[:120]:
    layer.trainable = False

# Compile Model with Optimized Learning Rate
model.compile(
    optimizer=Adam(learning_rate=5e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Add Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Train the Model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15,
    verbose=1,
    callbacks=[early_stopping]
)

# Unfreeze More Layers & Fine-tune
for layer in base_model.layers[80:]:
    layer.trainable = True

# **Implement Learning Rate Decay**
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-5,
    decay_steps=1000,
    decay_rate=0.95
)
optimizer = Adam(learning_rate=lr_schedule)

# Compile Again with Learning Rate Decay
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
history_fine = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    verbose=1,
    callbacks=[early_stopping]
)

# Evaluate on Test Set
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Final Test Accuracy: {test_accuracy:.2f}")

# Save Model
model.save('/content/drive/MyDrive/saved_models/efficientnet_model3.keras')


Copying dataset to Colab RAM...
Found 16460 files belonging to 2 classes.
Found 4668 files belonging to 2 classes.
Found 615 files belonging to 2 classes.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 362s 2s/step - accuracy: 0.7223 - loss: 7.6838 - val_accuracy: 0.5315 - val_loss: 5.4089
Epoch 2/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 252s 2s/step - accuracy: 0.8606 - loss: 3.5717 - val_accuracy: 0.8278 - val_loss: 2.1358
Epoch 3/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 252s 2s/step - accuracy: 0.9119 - loss: 1.6978 - val_accuracy: 0.9128 - val_loss: 1.1909
Epoch 4/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 252s 2s/step - accuracy: 0.9241 - loss: 1.0476 - val_accuracy: 0.9225 - val_loss: 0.8391
Epoch 5/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 252s 2s/step - accuracy: 0.9304 - loss: 0.7494 - val_accuracy: 0.9269 - val_loss: 0.6428
Epoch 6/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 252s 2s/step - accuracy: 0.9448 - loss: 0.5511 - val_accuracy: 0.9312 - val_loss: 0.5326
Epoch 7/15
129/129 ━

# Resnet50 model training for the augmented dataset of Channel shift method

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Normalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import os
import shutil

# Disable Mixed Precision (use float32 everywhere)
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('float32')

# Copy dataset from Google Drive to Colab RAM (Faster than streaming)
os.makedirs('/content/dataset', exist_ok=True)
if not os.path.exists('/content/dataset/train'):
    print("Copying dataset to Colab RAM...")
    shutil.copytree('/content/drive/MyDrive/train', '/content/dataset/train')
    shutil.copytree('/content/drive/MyDrive/val', '/content/dataset/val')
    shutil.copytree('/content/drive/MyDrive/test', '/content/dataset/test')

# Define dataset directories
train_dir = '/content/dataset/train'
val_dir = '/content/dataset/val'
test_dir = '/content/dataset/test'

# Define image dimensions and batch size
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 128
AUTOTUNE = tf.data.AUTOTUNE

# Load dataset using `tf.data`
raw_train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)
class_names = raw_train_dataset.class_names

# Define Data Augmentation Layer
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomBrightness(0.1),
])

# Define normalization layer
normalization_layer = Normalization()

# Function to convert images to float32
def cast_to_float32(image, label):
    return tf.cast(image, tf.float32), label

# Function to apply augmentation + normalization
def preprocess_image(image, label):
    image = data_augmentation(image)
    image = normalization_layer(image)
    return image, label

# Convert dataset images to float32 before applying normalization
normalized_train_dataset = raw_train_dataset.map(cast_to_float32)

# Adapt Normalization Layer using training data (only on images)
normalization_layer.adapt(normalized_train_dataset.map(lambda x, y: x))

# Apply augmentation & normalization to dataset
train_dataset = normalized_train_dataset.map(preprocess_image).prefetch(buffer_size=AUTOTUNE)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
).map(cast_to_float32).map(preprocess_image).prefetch(buffer_size=AUTOTUNE)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
).map(cast_to_float32).map(preprocess_image).prefetch(buffer_size=AUTOTUNE)

# Load Pre-trained ResNet50 with Correct Input Shape
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add Custom Classification Layers
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.005))(x)
x = BatchNormalization()(x)
x = Dropout(0.6)(x)  # Dropout Increased
x = Dense(256, activation='relu', kernel_regularizer=l2(0.005))(x)
x = Dropout(0.6)(x)

# Use `len(class_names)`
outputs = Dense(len(class_names), activation='softmax')(x)

# Define the Model
model = Model(inputs=base_model.input, outputs=outputs)

# Freeze the First 120 Layers (ResNet50 has ~175 layers)
for layer in base_model.layers[:120]:
    layer.trainable = False

# Compile Model with Optimized Learning Rate
model.compile(
    optimizer=Adam(learning_rate=5e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Add Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Train the Model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15,
    verbose=1,
    callbacks=[early_stopping]


# Unfreeze More Layers & Fine-tune
for layer in base_model.layers[80:]:
    layer.trainable = True

# **Implement Learning Rate Decay**
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-5,
    decay_steps=1000,
    decay_rate=0.95
)
optimizer = Adam(learning_rate=lr_schedule)

# Compile Again with Learning Rate Decay
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
history_fine = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    verbose=1,
    callbacks=[early_stopping]
)

# Evaluate on Test Set
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Final Test Accuracy: {test_accuracy:.2f}")

# Save Model
model.save('/content/drive/MyDrive/saved_models/resnet_model3.keras')


Copying dataset to Colab RAM...
Found 16460 files belonging to 2 classes.
Found 4668 files belonging to 2 classes.
Found 615 files belonging to 2 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 302s 2s/step - accuracy: 0.7327 - loss: 7.0522 - val_accuracy: 0.6433 - val_loss: 4.3755
Epoch 2/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 253s 2s/step - accuracy: 0.8534 - loss: 2.6641 - val_accuracy: 0.7644 - val_loss: 1.9237
Epoch 3/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 253s 2s/step - accuracy: 0.8870 - loss: 1.3659 - val_accuracy: 0.8203 - val_loss: 1.1061
Epoch 4/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 253s 2s/step - accuracy: 0.9051 - loss: 0.8460 - val_accuracy: 0.8912 - val_loss: 0.6832
Epoch 5/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 253s 2s/step - accuracy: 0.9147 - loss: 0.6133 - val_accuracy: 0.8920 - val_loss: 0.5495
Epoch 6/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 253s 2s/step - accuracy: 0.9315 - loss: 0.4538 - val_accuracy: 0.8888 - val_loss: 0.4911
Epoch 7/15
129/129 ━

# Resnet50 model training for the augmented dataset of Brightnes shift method

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Normalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import os
import shutil

# Disable Mixed Precision (use float32 everywhere)
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('float32')

# Copy dataset from Google Drive to Colab RAM (Faster than streaming)
os.makedirs('/content/dataset', exist_ok=True)
if not os.path.exists('/content/dataset/train'):
    print("Copying dataset to Colab RAM...")
    shutil.copytree('/content/drive/MyDrive/train', '/content/dataset/train')
    shutil.copytree('/content/drive/MyDrive/val', '/content/dataset/val')
    shutil.copytree('/content/drive/MyDrive/test', '/content/dataset/test')

# Define dataset directories
train_dir = '/content/dataset/train'
val_dir = '/content/dataset/val'
test_dir = '/content/dataset/test'

# Define image dimensions and batch size
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 128
AUTOTUNE = tf.data.AUTOTUNE

# Load dataset using `tf.data`
raw_train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)
class_names = raw_train_dataset.class_names

# Define Data Augmentation Layer
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomBrightness(0.1),
])

# Define normalization layer
normalization_layer = Normalization()

# Function to convert images to float32
def cast_to_float32(image, label):
    return tf.cast(image, tf.float32), label

# Function to apply augmentation + normalization
def preprocess_image(image, label):
    image = data_augmentation(image)
    image = normalization_layer(image)
    return image, label

# Convert dataset images to float32 before applying normalization
normalized_train_dataset = raw_train_dataset.map(cast_to_float32)

# Adapt Normalization Layer using training data (only on images)
normalization_layer.adapt(normalized_train_dataset.map(lambda x, y: x))

# Apply augmentation & normalization to dataset
train_dataset = normalized_train_dataset.map(preprocess_image).prefetch(buffer_size=AUTOTUNE)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
).map(cast_to_float32).map(preprocess_image).prefetch(buffer_size=AUTOTUNE)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
).map(cast_to_float32).map(preprocess_image).prefetch(buffer_size=AUTOTUNE)

# Load Pre-trained ResNet50 with Correct Input Shape
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add Custom Classification Layers
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.005))(x)
x = BatchNormalization()(x)
x = Dropout(0.6)(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.005))(x)
x = Dropout(0.6)(x)

# Use `len(class_names)`
outputs = Dense(len(class_names), activation='softmax')(x)

# Define the Model
model = Model(inputs=base_model.input, outputs=outputs)

# Freeze the First 120 Layers (ResNet50 has ~175 layers)
for layer in base_model.layers[:120]:
    layer.trainable = False

# Compile Model with Optimized Learning Rate
model.compile(
    optimizer=Adam(learning_rate=5e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Add Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Train the Model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15,
    verbose=1,
    callbacks=[early_stopping]
)

# Unfreeze More Layers & Fine-tune
for layer in base_model.layers[80:]:
    layer.trainable = True

# **Implement Learning Rate Decay**
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-5,
    decay_steps=1000,
    decay_rate=0.95
)
optimizer = Adam(learning_rate=lr_schedule)

# Compile Again with Learning Rate Decay
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
history_fine = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    verbose=1,
    callbacks=[early_stopping]
)

# Evaluate on Test Set
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Final Test Accuracy: {test_accuracy:.2f}")

# Save Model
model.save('/content/drive/MyDrive/saved_models/resnet_model4.keras')


Found 16460 files belonging to 2 classes.
Found 4668 files belonging to 2 classes.
Found 615 files belonging to 2 classes.
Epoch 1/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 289s 2s/step - accuracy: 0.6664 - loss: 7.2657 - val_accuracy: 0.6140 - val_loss: 7.0779
Epoch 2/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.7601 - loss: 3.0931 - val_accuracy: 0.7406 - val_loss: 1.9953
Epoch 3/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.8253 - loss: 1.6146 - val_accuracy: 0.6954 - val_loss: 1.3543
Epoch 4/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.8379 - loss: 1.0877 - val_accuracy: 0.7943 - val_loss: 0.9703
Epoch 5/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.8692 - loss: 0.8092 - val_accuracy: 0.8674 - val_loss: 0.7193
Epoch 6/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.8707 - loss: 0.6628 - val_accuracy: 0.8824 - val_loss: 0.5882
Epoch 7/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.8878 - loss: 0.5357 - val_accuracy: 

# EfficientnetB0 model training for the augmented dataset of Brightnes shift method

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Normalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import os
import shutil

# Disable Mixed Precision (use float32 everywhere)
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('float32')

# Copy dataset from Google Drive to Colab RAM (Faster than streaming)
os.makedirs('/content/dataset', exist_ok=True)
if not os.path.exists('/content/dataset/train'):
    print("Copying dataset to Colab RAM...")
    shutil.copytree('/content/drive/MyDrive/train', '/content/dataset/train')
    shutil.copytree('/content/drive/MyDrive/val', '/content/dataset/val')
    shutil.copytree('/content/drive/MyDrive/test', '/content/dataset/test')

# Define dataset directories
train_dir = '/content/dataset/train'
val_dir = '/content/dataset/val'
test_dir = '/content/dataset/test'

# Define image dimensions and batch size
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 128
AUTOTUNE = tf.data.AUTOTUNE

# Load dataset using `tf.data`
raw_train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)
class_names = raw_train_dataset.class_names

# Define Data Augmentation Layer
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomBrightness(0.1),
])

# Define normalization layer
normalization_layer = Normalization()

# Function to convert images to float32
def cast_to_float32(image, label):
    return tf.cast(image, tf.float32), label

# Function to apply augmentation + normalization
def preprocess_image(image, label):
    image = data_augmentation(image)
    image = normalization_layer(image)
    return image, label

# Convert dataset images to float32 before applying normalization
normalized_train_dataset = raw_train_dataset.map(cast_to_float32)

# Adapt Normalization Layer using training data (only on images)
normalization_layer.adapt(normalized_train_dataset.map(lambda x, y: x))

# Apply augmentation & normalization to dataset
train_dataset = normalized_train_dataset.map(preprocess_image).prefetch(buffer_size=AUTOTUNE)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
).map(cast_to_float32).map(preprocess_image).prefetch(buffer_size=AUTOTUNE)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
).map(cast_to_float32).map(preprocess_image).prefetch(buffer_size=AUTOTUNE)

# Load Pre-trained EfficientNetB0 with Correct Input Shape
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add Custom Classification Layers
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.005))(x)
x = BatchNormalization()(x)
x = Dropout(0.6)(x)  # Dropout Increased
x = Dense(256, activation='relu', kernel_regularizer=l2(0.005))(x)
x = Dropout(0.6)(x)

# Use `len(class_names)`
outputs = Dense(len(class_names), activation='softmax')(x)

# Define the Model
model = Model(inputs=base_model.input, outputs=outputs)

# Freeze the First 120 Layers (EfficientNetB0 has ~236 layers)
for layer in base_model.layers[:120]:
    layer.trainable = False

# Compile Model with Optimized Learning Rate
model.compile(
    optimizer=Adam(learning_rate=5e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Add Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Train the Model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15,
    verbose=1,
    callbacks=[early_stopping]
)

# Unfreeze More Layers & Fine-tune
for layer in base_model.layers[80:]:
    layer.trainable = True

# **Implement Learning Rate Decay**
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-5,
    decay_steps=1000,
    decay_rate=0.95
)
optimizer = Adam(learning_rate=lr_schedule)

# Compile Again with Learning Rate Decay
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
history_fine = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    verbose=1,
    callbacks=[early_stopping]
)

# Evaluate on Test Set
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Final Test Accuracy: {test_accuracy:.2f}")

# Save Model
model.save('/content/drive/MyDrive/saved_models/efficientnet_model4.keras')


Copying dataset to Colab RAM...
Found 16460 files belonging to 2 classes.
Found 4668 files belonging to 2 classes.
Found 615 files belonging to 2 classes.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 357s 2s/step - accuracy: 0.6710 - loss: 7.9658 - val_accuracy: 0.5570 - val_loss: 5.3012
Epoch 2/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.8095 - loss: 4.2413 - val_accuracy: 0.7817 - val_loss: 2.6021
Epoch 3/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.8591 - loss: 2.1416 - val_accuracy: 0.8380 - val_loss: 1.5438
Epoch 4/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.8782 - loss: 1.3423 - val_accuracy: 0.8948 - val_loss: 1.0308
Epoch 5/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.8995 - loss: 0.9359 - val_accuracy: 0.8734 - val_loss: 0.8402
Epoch 6/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.9107 - loss: 0.7515 - val_accuracy: 0.9180 - val_loss: 0.6348
Epoch 7/15
129/129 ━

# Resnet50 model training for the augmented dataset of both Brightnes and random channel

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Normalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import os
import shutil

# Disable Mixed Precision (use float32 everywhere)
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('float32')

# Copy dataset from Google Drive to Colab RAM (Faster than streaming)
os.makedirs('/content/dataset', exist_ok=True)
if not os.path.exists('/content/dataset/train'):
    print("Copying dataset to Colab RAM...")
    shutil.copytree('/content/drive/MyDrive/train', '/content/dataset/train')
    shutil.copytree('/content/drive/MyDrive/val', '/content/dataset/val')
    shutil.copytree('/content/drive/MyDrive/test', '/content/dataset/test')

# Define dataset directories
train_dir = '/content/dataset/train'
val_dir = '/content/dataset/val'
test_dir = '/content/dataset/test'

# Define image dimensions and batch size
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 128
AUTOTUNE = tf.data.AUTOTUNE

# Load dataset using `tf.data`
raw_train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)
class_names = raw_train_dataset.class_names

# Define Data Augmentation Layer
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomBrightness(0.1),
])

# Define normalization layer
normalization_layer = Normalization()

# Function to convert images to float32
def cast_to_float32(image, label):
    return tf.cast(image, tf.float32), label

# Function to apply augmentation + normalization
def preprocess_image(image, label):
    image = data_augmentation(image)
    image = normalization_layer(image)
    return image, label

# Convert dataset images to float32 before applying normalization
normalized_train_dataset = raw_train_dataset.map(cast_to_float32)

# Adapt Normalization Layer using training data (only on images)
normalization_layer.adapt(normalized_train_dataset.map(lambda x, y: x))

# Apply augmentation & normalization to dataset
train_dataset = normalized_train_dataset.map(preprocess_image).prefetch(buffer_size=AUTOTUNE)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
).map(cast_to_float32).map(preprocess_image).prefetch(buffer_size=AUTOTUNE)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
).map(cast_to_float32).map(preprocess_image).prefetch(buffer_size=AUTOTUNE)

# Load Pre-trained ResNet50 with Correct Input Shape
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add Custom Classification Layers
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.005))(x)
x = BatchNormalization()(x)
x = Dropout(0.6)(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.005))(x)
x = Dropout(0.6)(x)

# Use `len(class_names)`
outputs = Dense(len(class_names), activation='softmax')(x)

# Define the Model
model = Model(inputs=base_model.input, outputs=outputs)

# Freeze the First 120 Layers (ResNet50 has ~175 layers)
for layer in base_model.layers[:120]:
    layer.trainable = False

# Compile Model with Optimized Learning Rate
model.compile(
    optimizer=Adam(learning_rate=5e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Add Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Train the Model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15,
    verbose=1,
    callbacks=[early_stopping]
)

# Unfreeze More Layers & Fine-tune
for layer in base_model.layers[80:]:
    layer.trainable = True

# **Implement Learning Rate Decay**
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-5,
    decay_steps=1000,
    decay_rate=0.95
)
optimizer = Adam(learning_rate=lr_schedule)

# Compile Again with Learning Rate Decay
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
history_fine = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    verbose=1,
    callbacks=[early_stopping]
)

# Evaluate on Test Set
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Final Test Accuracy: {test_accuracy:.2f}")

# Save Model
model.save('/content/drive/MyDrive/saved_models/resnet_model5.keras')


Copying dataset to Colab RAM...
Found 16460 files belonging to 2 classes.
Found 4668 files belonging to 2 classes.
Found 615 files belonging to 2 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 301s 2s/step - accuracy: 0.6666 - loss: 7.2984 - val_accuracy: 0.6144 - val_loss: 4.4118
Epoch 2/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 252s 2s/step - accuracy: 0.7850 - loss: 3.0645 - val_accuracy: 0.7791 - val_loss: 2.0329
Epoch 3/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 252s 2s/step - accuracy: 0.8236 - loss: 1.6971 - val_accuracy: 0.8085 - val_loss: 1.2499
Epoch 4/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.8479 - loss: 1.1219 - val_accuracy: 0.8436 - val_loss: 0.9282
Epoch 5/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.8652 - loss: 0.8314 - val_accuracy: 0.8179 - val_loss: 0.8256
Epoch 6/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 252s 2s/step - accuracy: 0.8793 - loss: 0.6910 - val_accuracy: 0.8380 - val_loss: 0.6757
Epoch 7/15
129/129 ━

# EfficientnetB0 model training for the augmented dataset of both Brightnes and random channel

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Normalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import os
import shutil

# Disable Mixed Precision (use float32 everywhere)
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('float32')

# Copy dataset from Google Drive to Colab RAM (Faster than streaming)
os.makedirs('/content/dataset', exist_ok=True)
if not os.path.exists('/content/dataset/train'):
    print("Copying dataset to Colab RAM...")
    shutil.copytree('/content/drive/MyDrive/train', '/content/dataset/train')
    shutil.copytree('/content/drive/MyDrive/val', '/content/dataset/val')
    shutil.copytree('/content/drive/MyDrive/test', '/content/dataset/test')

# Define dataset directories
train_dir = '/content/dataset/train'
val_dir = '/content/dataset/val'
test_dir = '/content/dataset/test'

# Define image dimensions and batch size
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 128
AUTOTUNE = tf.data.AUTOTUNE

# Load dataset using `tf.data`
raw_train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)
class_names = raw_train_dataset.class_names

# Define Data Augmentation Layer
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomBrightness(0.1),
])

# Define normalization layer
normalization_layer = Normalization()

# Function to convert images to float32
def cast_to_float32(image, label):
    return tf.cast(image, tf.float32), label

# Function to apply augmentation + normalization
def preprocess_image(image, label):
    image = data_augmentation(image)
    image = normalization_layer(image)
    return image, label

# Convert dataset images to float32 before applying normalization
normalized_train_dataset = raw_train_dataset.map(cast_to_float32)

# Adapt Normalization Layer using training data (only on images)
normalization_layer.adapt(normalized_train_dataset.map(lambda x, y: x))

# Apply augmentation & normalization to dataset
train_dataset = normalized_train_dataset.map(preprocess_image).prefetch(buffer_size=AUTOTUNE)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
).map(cast_to_float32).map(preprocess_image).prefetch(buffer_size=AUTOTUNE)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
).map(cast_to_float32).map(preprocess_image).prefetch(buffer_size=AUTOTUNE)

# Load Pre-trained EfficientNetB0 with Correct Input Shape
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add Custom Classification Layers
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.005))(x)
x = BatchNormalization()(x)
x = Dropout(0.6)(x)  # Dropout Increased
x = Dense(256, activation='relu', kernel_regularizer=l2(0.005))(x)
x = Dropout(0.6)(x)

# Use `len(class_names)`
outputs = Dense(len(class_names), activation='softmax')(x)

# Define the Model
model = Model(inputs=base_model.input, outputs=outputs)

# Freeze the First 120 Layers (EfficientNetB0 has ~236 layers)
for layer in base_model.layers[:120]:
    layer.trainable = False

# Compile Model with Optimized Learning Rate
model.compile(
    optimizer=Adam(learning_rate=5e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Add Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Train the Model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15,
    verbose=1,
    callbacks=[early_stopping]
)

# Unfreeze More Layers & Fine-tune
for layer in base_model.layers[80:]:
    layer.trainable = True

# **Implement Learning Rate Decay**
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-5,
    decay_steps=1000,
    decay_rate=0.95
)
optimizer = Adam(learning_rate=lr_schedule)

# Compile Again with Learning Rate Decay
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
history_fine = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    verbose=1,
    callbacks=[early_stopping]
)

# Evaluate on Test Set
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Final Test Accuracy: {test_accuracy:.2f}")

# Save Model
model.save('/content/drive/MyDrive/saved_models/efficientnet_model5.keras')


Found 16460 files belonging to 2 classes.
Found 4668 files belonging to 2 classes.
Found 615 files belonging to 2 classes.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 354s 2s/step - accuracy: 0.6548 - loss: 8.0239 - val_accuracy: 0.5281 - val_loss: 5.6476
Epoch 2/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.7944 - loss: 4.2322 - val_accuracy: 0.6050 - val_loss: 3.1446
Epoch 3/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.8402 - loss: 2.2439 - val_accuracy: 0.8558 - val_loss: 1.5711
Epoch 4/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.8650 - loss: 1.4007 - val_accuracy: 0.9002 - val_loss: 1.0208
Epoch 5/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.8917 - loss: 0.9689 - val_accuracy: 0.8342 - val_loss: 0.9583
Epoch 6/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.8999 - loss: 0.7577 - val_accuracy: 0.9147 - val_loss: 0.6378
Epoch 7/15
129/129 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step